In [2]:
import os
%pwd
os.chdir("../")

In [3]:
import os
from box.exceptions import BoxValueError
import yaml

import json
import joblib
from ensure import ensure_annotations
from box import ConfigBox
from pathlib import Path
from typing import Any
import base64
from pathlib import Path

In [4]:
def create_directories(path_to_dir:list,verbose=True):
    for path in path_to_dir:
        os.makedirs(path,exist_ok=True)
       # if verbose:
           # logger.info(f"created dierctor ast {path}")


In [5]:
def read_yaml(path_to_yaml:Path)-> ConfigBox:
    try:
        with open(path_to_yaml) as yaml_file:
            content=yaml.safe_load(yaml_file)
           # logger.info(f"yaml file {path_to_yaml} load sucessfuly")
            return ConfigBox(content)
    except BoxValueError:
        raise ValueError("yaml file is empty")
    except Exception as e:
        raise(e)

In [6]:
from dataclasses import dataclass
from pathlib import Path
@dataclass
class Datatrainingconfig:
    root_dir:Path
    source_name: str
    transformed_path: Path
    saving_model: Path

In [18]:
class ConfigurationManger:
    def __init__(self,config_file_path,params_file_path):
        self.config=read_yaml(config_file_path)
        self.params=read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])

    def get_data_training_config(self) -> Datatrainingconfig:
        config=self.config.data_training

        create_directories([config.root_dir])
        data_training_config=Datatrainingconfig(


            root_dir=config.root_dir,


            
            source_name= config.source_name,
           
           
            transformed_path=config.transformed_path,
            saving_model=config.saving_model

                
        )
      
        return data_training_config

In [25]:
from datasets import load_from_disk , DatasetDict
from transformers import AutoModelForSequenceClassification , Trainer, TrainingArguments, DataCollatorWithPadding ,AutoTokenizer


In [ ]:
from sklearn.model_selection import train_test_split

In [26]:

class Data_training:
    def __init__(self, config:Datatrainingconfig):
        self.config=config 
        self.dataset=load_from_disk(self.config.transformed_path)
        self.model = AutoModelForSequenceClassification.from_pretrained(self.config.source_name,num_labels=5)
        
    def training_args(self):
        
        tokenized_datasets=self.dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
        train_testvalid = self.dataset['train'].train_test_split(test_size=0.1)
        train_test = train_testvalid['train'].train_test_split(test_size=0.1)
        tokenized_datasets = DatasetDict({
        'train': train_test['train'],
        'test': train_testvalid['test'],
        'valid': train_test['test']
})
        train_dataset = tokenized_datasets['train']
        valid_dataset = tokenized_datasets['valid']
        test_dataset = tokenized_datasets['test']  
        tokenizer = AutoTokenizer.from_pretrained(self.config.source_name) 

        data_collator = DataCollatorWithPadding(tokenizer)
        training_args = TrainingArguments(
        output_dir='./results',          # output directory
        num_train_epochs=3,              # number of training epochs
        per_device_train_batch_size=8,   # batch size for training
        per_device_eval_batch_size=8,    # batch size for evaluation
        warmup_steps=500,                # number of warmup steps for learning rate scheduler
        weight_decay=0.01,               # strength of weight decay
        logging_dir='./logs',            # directory for storing logs
        logging_steps=10,
        evaluation_strategy='epoch'      # Evaluate every epoch
    )
        trainer = Trainer(
        model=self.model,                         # the instantiated 🤗 Transformers model to be trained
        args=training_args,                  # training arguments, defined above
        train_dataset=train_dataset,         # training dataset
        eval_dataset=valid_dataset,          # evaluation dataset
        data_collator=data_collator          # data collator
    )
        trainer.train()

        validation_results = trainer.evaluate(eval_dataset=valid_dataset)
        print("Validation results:", validation_results)

        # Evaluate the model on the test set
        test_results = trainer.evaluate(eval_dataset=test_dataset)
        print("Test results:", test_results)

        model_save_path = "./"
        self.model.save_pretrained(self.config.saving_model)
        tokenizer.save_pretrained(self.config.saving_model)


In [27]:
try:
    config=ConfigurationManger(config_file_path=Path("config.yaml"),params_file_path=Path("params.yaml"))
    datatraining_config=config.get_data_training_config()
    data_training=Data_training(config=datatraining_config)

    
    data_training.training_args()
except Exception as e:
    raise e

  0%|          | 3/87501 [00:14<110:11:40,  4.53s/it]

KeyboardInterrupt: 